# Spotify data downloading

To download more information Spotify API was used. All the downloaded data was saved in the MongoDB collection on local machine.

In [1]:
import spotipy
import spotipy.oauth2 as oauth2
import spotipy.util as util
from IPython.display import clear_output
from pymongo import MongoClient
import entity
import time

## Establish connection

First, to be able to use Spotify API, some additional values must have been set. Each application must be registered and before each time it needs to start sending some requests, special token must be created. This token must be refreshed after some time.

In [2]:
SPOTIPY_CLIENT_ID = 'd70e66cbc22e4e1287c50c97b54c2457'
SPOTIPY_CLIENT_SECRET = 'e20d58cae494486392723da75a3ef393'
SPOTIPY_REDIRECT_URI = 'http://localhost/?code=...'

credentials = oauth2.SpotifyClientCredentials(
client_id="d70e66cbc22e4e1287c50c97b54c2457",
client_secret="e20d58cae494486392723da75a3ef393")

token = credentials.get_access_token()
print(token)
spotify = spotipy.Spotify(auth=token)

db_client = MongoClient('localhost', 27017)
db = db_client.musicdata
collection = db.get_collection("artist_spotify")

BQCX73M5zDy2OeQRJCD45Pxeawa8nmrXkEoLXZtFjhyXs8eIWKobQiP2rwOozYtXDSnZciC8g9kD5SOucFY


## Download artists using Spotify IDs

First, data from Spotify was downloaded using existing Spotify IDs.

In [2]:
artist_list =[]
not_fetched = [] #because of some errors - try to fetch each entry separately
keys = ['followers','genres','id','name', 'popularity']
total_artists = 0
for artist in collection.find():
    if len(artist['_id']) == 22:
        artist_list.append(artist['_id'])
        total_artists +=1
    if len(artist_list) >= 50:
        print(total_artists)
        try:
            result = spotify.artists(artist_list)
            #print(result)
            artist_list = []
            for art in result['artists']:
                if art != None:
                    new_art = {k:v for k, v in art.items() if k in keys}
                    new_art['_id'] = new_art['id']
                    new_art.pop('id')
                    #collection.update_one({'_id':new_art['_id']}, {"$set": new_art}, upsert=True)
        except spotipy.client.SpotifyException as err:
            if err.code == 401:
                print("Spotipy 401 error:", err.msg)
                time.sleep(60)
            else:
                print("Spotipy error:", err.code, err.http_status, err.msg)
                not_fetched.append(artist_list)
                artist_list = []
                continue

50
100
150
200
250
300
350
400
450
500
Spotipy error: -1 400 https://api.spotify.com/v1/artists/?ids=5sY7DOqW3OxgYzVr4DjQiV,0wLCpPj7FYTGxZZwpzRogz,2dfDjeZroUd3LWmSFrAZCD,6Uk2pq20qpkftbU2PEDWoU,73Bdg5wnkiWMtLKeK5mGIn,1yCvWl3rxWT6TnZMPBE0i0,5svDD0GKNeaBBHCcuP3b3k,0UZ0eyqZFeuyuDWtGipr9x,3peZFZBRhRGd1lTTsNmaPM,6slyJwua5e5GuhuP82sTHR,2GDqxAIoVhLMmtFP0NrUlA,5lSwtyrStj82mnFdcF2yfb,6flBUmmOMLNhD4EJhGwgpG,3eIJEraAUU6QhYCQaD4kJ4,2tGBz8Jzc81v4tRPmTs8eq,1gEqsqT3mT13pTcUhEyzzs,1FPk3Fo5EyBYOxEiT2V0jP,0WZOmdnCln6FK6GM9e2tGm,4mCLngnQNRBnIqIZW8BEnJ,5n3dkXwwvC1w42hp5U0Qxq,64RGS9OBGsUWkopEtO5Cz7,014v7wyYEzOsxoWbiqXFOA,4dYQmk5ma04mZ1KJ9KkAQK,3OFJwVl0Y6ceUXMvc1SFHC,3ZmRPXhH95QiWYqIO1esbC,0dlr1FsAcalNnpq1q2RIoT,6oLmtHnZd8alkjhUP9dBzo,5TGWjahgYwvJvz1HtugJDG,3puJQaJutGEPvjOkbGKN8m,30Ns8wCbxSPz2L1tnSypTG,2ZfHMUPrxoZNDTBPHmmdhj,7cWnPCnFKYKg5W0JKaf118,5HePAITZSCxQ0H9PEKZ5DH,230OCOfn3VjbOdmSYiiODP,3pO5VjZ4wOHCMBXOvbMISG,1wXinjMsxsHJGk7wi3wYsu,5L5T9sLlIgtVQx4KAdB0q1,4WoaKwcuPbrlwMfZvOazNh,2hzYUcjBdSxYsV3U51RITC,0d

8950
9000
9050
9100
9150
9200
9250
9300
9350
Spotipy error: -1 400 https://api.spotify.com/v1/artists/?ids=6gVwdzZkHpqp92RHvjqXys,5lZfMMtoleaZIOJ7HqnPK0,1W3sZMluDmealiI6B5awE9,2hTnSPLuF54jseC35e210D,2JI8ViuZDBybY6Xd9ujUrb,757F6VRU2lCiUhqrRHmt8Y,0jAsgQcw1SQgsSkz8CczHh,0kMLg1Nu0mtNs85irGJxmy,2tXXoTykabsZNETr1Sr1zC,6pM5NZalHT5nD9vDEz2Ukx,6fgxuUxhD8jiPKq21mvIoo,5bfw7ydBnntO3NbtHagaQf,4rGGVLUW4jKwW7HVwTr40U,3A94tOOlaFL73w58WjhTrh,4Xux8HKTsa33WmP86n47n7,6JQETbp0JAmQr26FZjSqJd,3MXhtYDNuzQQmLfOKFgPiI,78YqeIji3mgAS2K1Maca6x,7nstqHP5wRNlaUeSByC86I,7Fph7U6qidZ2E97xKKsD4m,7qs9lBPZgZzLAYAWfHLAll,4rYCMf7ApPRVG3yjCbxJ8m,0AFbebB7EpLm1qzqBRwyWt,17ZmsQR8Jv0dolI7uNBiaT,6AmX6uFptshw56Z5LbOFhZ,60Ygpctc1ZdaRuvvrkiqAU,11RgIVDXeEQH3QiYcslcSW,5cz9CTjiVooDMiVFPmLJTx,1aCLiYz6Kl83key0zcgCxc,1p6wR69pnH9LBWZvwliuz2,2gQKtXD5BZqKT8Tl9JfFcy,3HmzM7c3S5iM1jBG3gl0FF,62TD7509VQIxUe4WpwO0s3,4bw2Am3p9ji3mYsXNXtQcd,1mZm40boQmdGKicfbNkd0r,76X8p2EEoEJmDhgzLExcbZ,?url=https%3A%2F%2Fope,6XZxFuZaBnZBngpxbut80O,1wyuV2UwDaVUPcvuC8Y

In [6]:
# Because of some errors/exceptions some of the artists were not fetched - run same code as before again but
# only for missing entries.
flat_list = [item for sublist in not_fetched for item in sublist]
for artist in flat_list:
    try:
        result = spotify.artists([artist])
        #print(result)
        art = result['artists'][0]
        #print(art)
        if art != None:
            new_art = {k:v for k, v in art.items() if k in keys}
            new_art['_id'] = new_art['id']
            new_art.pop('id')
            #print(new_art)
            collection.update_one({'_id':new_art['_id']}, {"$set": new_art}, upsert=True)
    except spotipy.client.SpotifyException as err:
        if err.code == 401:
            print("Spotipy 401 error:", err.msg)
            time.sleep(60)
        else:
            print("Spotipy error:", err.code, err.http_status, err.msg)
       

Spotipy error: -1 400 https://api.spotify.com/v1/artists/?ids=artist%3ALost%20Childr:
 invalid id
Spotipy error: -1 400 https://api.spotify.com/v1/artists/?ids=Andreas%20Rodlund%20%2:
 error
Spotipy error: -1 400 https://api.spotify.com/v1/artists/?ids=shibbydingfizz?play=tr:
 invalid id
Spotipy error: -1 400 https://api.spotify.com/v1/artists/?ids=epicrapbattles?play=tr:
 invalid id
Spotipy error: -1 400 https://api.spotify.com/v1/artists/?ids=url?sa=t&rct=j&q=&esrc:
 invalid id
Spotipy error: -1 400 https://api.spotify.com/v1/artists/?ids=1250922082?play=true&u:
 invalid id
Spotipy error: -1 400 https://api.spotify.com/v1/artists/?ids=:
 invalid id
Spotipy error: -1 400 https://api.spotify.com/v1/artists/?ids=?url=https%3A%2F%2Fope:
 invalid id
Spotipy error: -1 400 https://api.spotify.com/v1/artists/?ids=de%20b-kant?play=true&:
 invalid id
Spotipy error: -1 400 https://api.spotify.com/v1/artists/?ids=1216703655?play=true&u:
 invalid id
Spotipy error: -1 400 https://api.spotify.com/v

## Download artists using artists' names from Musicbrainz

For the rest of artists from Musicbrainz different approach must have been used. Artists' information was downloaded using existing names of artists in data downloaded from Musicbrainz. Althought, by this approach a lot of new artists were fetched, not all of them have direct mapping between those two databases. The reason for that is for each Musicbrainz name, more than one artist from Spotify were downloaded. Merging of those two datasets is described here: TODO - add link to explanation of the merging process

In [3]:
musicbrainz_collection = db.get_collection("artists_mb")
spotify_by_names_collection = db.get_collection("artists_spotify_bynames")

In [7]:
keys = ['followers','genres','id','name', 'popularity']
id_mb = 0

for mb_artist in musicbrainz_collection.find({},no_cursor_timeout=True)[830000:]:
    clear_output(wait=True)
    print(id_mb)
    id_mb +=1
    mb_name = mb_artist['name']
    mb_id = mb_artist['_id']
    mb_info = {'mb_name': mb_name, 'mb_id': mb_id}
    result = {}
    try:
        result = spotify.search(q=mb_artist['name'],type='artist', limit=5)
    except spotipy.client.SpotifyException as err:
        if err.code == 401:
            print("Spotipy 401 error:", err.msg)
            time.sleep(60)
            continue
        else:
            print("Spotipy error:", err.code, err.http_status, err.msg)
            token = credentials.get_access_token()
            print(token)
            spotify = spotipy.Spotify(auth=token)
            continue
    except Exception as err:
        print("Exception {1}", err)
        time.sleep(20)
        continue
    #print(result)   
    if result['artists']['total']>0:
        for found_artist in result['artists']['items']:
            new_art = {k:v for k, v in found_artist.items() if k in keys}
            new_art['_id'] = new_art['id']
            new_art.pop('id')
            mb_info_list = []

            artist_already_in_db = spotify_by_names_collection.find_one({"_id": new_art['_id']})
            if artist_already_in_db != None:
                mb_info_list = artist_already_in_db['mb_info']
                mb_info_list.append(mb_info)
                new_art['mb_info'] = mb_info_list
            else:
                new_art['mb_info'] = [mb_info]
             
            spotify_by_names_collection.update_one({'_id':new_art['_id']}, {"$set": new_art}, upsert=True)
        

488076
